# Settings

In [25]:
import unicodedata
import re

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
import math
import itertools
import os

In [8]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

# Preprocessing data for training

In [2]:
corpus_name = "cornell movie-dialogs corpus"
datafile = "../../cornell movie-dialogs corpus/formatted_movie_lines.txt"

## Create word library

In [3]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [4]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('=')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus_name, datafile):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
#save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus_name, datafile)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64201 sentence pairs
Counting words...
Counted words: 17991

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


## Filter sentences that include words that occur less than 3 times in data

In [5]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7820 / 17988 = 0.4347
Trimmed from 64201 pairs to 53110, 0.8272 of total


# Create batch tensors with padding

In [12]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 100,    4,   92,   34, 5445],
        [  68,    4,    7,   66,    2],
        [   7, 6749,  544,    2,    0],
        [4355,    4,    6,    0,    0],
        [   6,    2,    2,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([6, 5, 5, 3, 2])
target_variable: tensor([[  34,   25,  147,  318,  318],
        [  66,  534,  582,   66, 5446],
        [   2, 2330,    8,    2,    6],
        [   0,  159,    7,    0,    2],
        [   0,    4,   44,    0,    0],
        [   0,    2,  594,    0,    0],
        [   0,    0,    6,    0,    0],
        [   0,    0,    2,    0,    0]])
mask: tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 0, 1],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0]], dtype=torch.uint8)
max_target_len: 8


# Create model structure

Seq2seq bidirectional model with attention. 

## Encoder

In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

## Decoder

### Attention class

In [14]:
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

### Attention decoder

In [15]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Training procedure

### Los function

In [16]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

### Train methods

In [17]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [18]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Evaluation

In [19]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [20]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc, sentence):
    input_sentence = ''
    try:
        # Get input sentence
        input_sentence = sentence
        # Normalize sentence
        input_sentence = normalizeString(input_sentence)
        # Evaluate sentence
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
        # Format and print response sentence
        output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        print('Bot:', ' '.join(output_words))

    except KeyError:
        print("Error: Encountered unknown word.")

# Run model

In [21]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 50
encoder_n_layers = 1
decoder_n_layers = 1
dropout = 0.1
batch_size = 16

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


## Run training

In [26]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 1000
print_every = 1
save_every = 500
save_dir = "../trained_model_weights"

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.1%; Average loss: 4.6716
Iteration: 2; Percent complete: 0.2%; Average loss: 4.4210
Iteration: 3; Percent complete: 0.3%; Average loss: 5.0366
Iteration: 4; Percent complete: 0.4%; Average loss: 4.3985
Iteration: 5; Percent complete: 0.5%; Average loss: 4.4543
Iteration: 6; Percent complete: 0.6%; Average loss: 4.6838
Iteration: 7; Percent complete: 0.7%; Average loss: 4.6881
Iteration: 8; Percent complete: 0.8%; Average loss: 4.7883
Iteration: 9; Percent complete: 0.9%; Average loss: 4.6528
Iteration: 10; Percent complete: 1.0%; Average loss: 4.9286
Iteration: 11; Percent complete: 1.1%; Average loss: 4.7375
Iteration: 12; Percent complete: 1.2%; Average loss: 4.2770
Iteration: 13; Percent complete: 1.3%; Average loss: 4.8923
Iteration: 14; Percent complete: 1.4%; Average loss: 4.9650
Iteration: 15; Percent complete: 1.5%; Average loss: 4.7650
Iteration: 16; Percent complete: 1.6%

Iteration: 138; Percent complete: 13.8%; Average loss: 4.7430
Iteration: 139; Percent complete: 13.9%; Average loss: 4.1790
Iteration: 140; Percent complete: 14.0%; Average loss: 4.3370
Iteration: 141; Percent complete: 14.1%; Average loss: 4.5409
Iteration: 142; Percent complete: 14.2%; Average loss: 4.8177
Iteration: 143; Percent complete: 14.3%; Average loss: 4.1640
Iteration: 144; Percent complete: 14.4%; Average loss: 4.1322
Iteration: 145; Percent complete: 14.5%; Average loss: 4.7863
Iteration: 146; Percent complete: 14.6%; Average loss: 4.8631
Iteration: 147; Percent complete: 14.7%; Average loss: 4.3964
Iteration: 148; Percent complete: 14.8%; Average loss: 4.5776
Iteration: 149; Percent complete: 14.9%; Average loss: 4.6630
Iteration: 150; Percent complete: 15.0%; Average loss: 4.2220
Iteration: 151; Percent complete: 15.1%; Average loss: 4.7230
Iteration: 152; Percent complete: 15.2%; Average loss: 4.5036
Iteration: 153; Percent complete: 15.3%; Average loss: 4.6157
Iteratio

Iteration: 274; Percent complete: 27.4%; Average loss: 4.8919
Iteration: 275; Percent complete: 27.5%; Average loss: 4.4696
Iteration: 276; Percent complete: 27.6%; Average loss: 4.8342
Iteration: 277; Percent complete: 27.7%; Average loss: 4.7440
Iteration: 278; Percent complete: 27.8%; Average loss: 3.9804
Iteration: 279; Percent complete: 27.9%; Average loss: 4.5218
Iteration: 280; Percent complete: 28.0%; Average loss: 4.5181
Iteration: 281; Percent complete: 28.1%; Average loss: 4.6018
Iteration: 282; Percent complete: 28.2%; Average loss: 4.4116
Iteration: 283; Percent complete: 28.3%; Average loss: 4.5451
Iteration: 284; Percent complete: 28.4%; Average loss: 4.1842
Iteration: 285; Percent complete: 28.5%; Average loss: 4.8458
Iteration: 286; Percent complete: 28.6%; Average loss: 4.7217
Iteration: 287; Percent complete: 28.7%; Average loss: 4.7976
Iteration: 288; Percent complete: 28.8%; Average loss: 4.6714
Iteration: 289; Percent complete: 28.9%; Average loss: 4.4460
Iteratio

Iteration: 409; Percent complete: 40.9%; Average loss: 4.5076
Iteration: 410; Percent complete: 41.0%; Average loss: 4.7455
Iteration: 411; Percent complete: 41.1%; Average loss: 4.4083
Iteration: 412; Percent complete: 41.2%; Average loss: 4.3769
Iteration: 413; Percent complete: 41.3%; Average loss: 4.7877
Iteration: 414; Percent complete: 41.4%; Average loss: 4.5975
Iteration: 415; Percent complete: 41.5%; Average loss: 4.5216
Iteration: 416; Percent complete: 41.6%; Average loss: 4.7290
Iteration: 417; Percent complete: 41.7%; Average loss: 4.7804
Iteration: 418; Percent complete: 41.8%; Average loss: 4.4405
Iteration: 419; Percent complete: 41.9%; Average loss: 4.5272
Iteration: 420; Percent complete: 42.0%; Average loss: 4.5304
Iteration: 421; Percent complete: 42.1%; Average loss: 4.6875
Iteration: 422; Percent complete: 42.2%; Average loss: 4.7486
Iteration: 423; Percent complete: 42.3%; Average loss: 4.3293
Iteration: 424; Percent complete: 42.4%; Average loss: 4.2765
Iteratio

Iteration: 545; Percent complete: 54.5%; Average loss: 4.6292
Iteration: 546; Percent complete: 54.6%; Average loss: 4.1114
Iteration: 547; Percent complete: 54.7%; Average loss: 4.1687
Iteration: 548; Percent complete: 54.8%; Average loss: 4.8593
Iteration: 549; Percent complete: 54.9%; Average loss: 3.8458
Iteration: 550; Percent complete: 55.0%; Average loss: 4.4842
Iteration: 551; Percent complete: 55.1%; Average loss: 4.3273
Iteration: 552; Percent complete: 55.2%; Average loss: 4.4739
Iteration: 553; Percent complete: 55.3%; Average loss: 4.0263
Iteration: 554; Percent complete: 55.4%; Average loss: 4.5344
Iteration: 555; Percent complete: 55.5%; Average loss: 4.4706
Iteration: 556; Percent complete: 55.6%; Average loss: 4.1953
Iteration: 557; Percent complete: 55.7%; Average loss: 4.4627
Iteration: 558; Percent complete: 55.8%; Average loss: 4.3367
Iteration: 559; Percent complete: 55.9%; Average loss: 4.6579
Iteration: 560; Percent complete: 56.0%; Average loss: 4.5395
Iteratio

Iteration: 679; Percent complete: 67.9%; Average loss: 3.9428
Iteration: 680; Percent complete: 68.0%; Average loss: 4.2858
Iteration: 681; Percent complete: 68.1%; Average loss: 4.5696
Iteration: 682; Percent complete: 68.2%; Average loss: 4.7795
Iteration: 683; Percent complete: 68.3%; Average loss: 4.4161
Iteration: 684; Percent complete: 68.4%; Average loss: 4.5601
Iteration: 685; Percent complete: 68.5%; Average loss: 4.3385
Iteration: 686; Percent complete: 68.6%; Average loss: 4.5655
Iteration: 687; Percent complete: 68.7%; Average loss: 4.3604
Iteration: 688; Percent complete: 68.8%; Average loss: 4.1639
Iteration: 689; Percent complete: 68.9%; Average loss: 4.6318
Iteration: 690; Percent complete: 69.0%; Average loss: 4.4469
Iteration: 691; Percent complete: 69.1%; Average loss: 4.2822
Iteration: 692; Percent complete: 69.2%; Average loss: 4.1949
Iteration: 693; Percent complete: 69.3%; Average loss: 4.1983
Iteration: 694; Percent complete: 69.4%; Average loss: 4.7555
Iteratio

Iteration: 815; Percent complete: 81.5%; Average loss: 4.5731
Iteration: 816; Percent complete: 81.6%; Average loss: 4.1845
Iteration: 817; Percent complete: 81.7%; Average loss: 4.4873
Iteration: 818; Percent complete: 81.8%; Average loss: 4.2270
Iteration: 819; Percent complete: 81.9%; Average loss: 4.3022
Iteration: 820; Percent complete: 82.0%; Average loss: 4.7253
Iteration: 821; Percent complete: 82.1%; Average loss: 4.2501
Iteration: 822; Percent complete: 82.2%; Average loss: 4.4545
Iteration: 823; Percent complete: 82.3%; Average loss: 3.8662
Iteration: 824; Percent complete: 82.4%; Average loss: 4.3132
Iteration: 825; Percent complete: 82.5%; Average loss: 3.9903
Iteration: 826; Percent complete: 82.6%; Average loss: 4.1422
Iteration: 827; Percent complete: 82.7%; Average loss: 4.3848
Iteration: 828; Percent complete: 82.8%; Average loss: 4.9478
Iteration: 829; Percent complete: 82.9%; Average loss: 4.6190
Iteration: 830; Percent complete: 83.0%; Average loss: 4.6757
Iteratio

Iteration: 950; Percent complete: 95.0%; Average loss: 4.5766
Iteration: 951; Percent complete: 95.1%; Average loss: 4.5332
Iteration: 952; Percent complete: 95.2%; Average loss: 4.0636
Iteration: 953; Percent complete: 95.3%; Average loss: 3.7641
Iteration: 954; Percent complete: 95.4%; Average loss: 4.4666
Iteration: 955; Percent complete: 95.5%; Average loss: 4.5183
Iteration: 956; Percent complete: 95.6%; Average loss: 4.5666
Iteration: 957; Percent complete: 95.7%; Average loss: 4.2133
Iteration: 958; Percent complete: 95.8%; Average loss: 4.3520
Iteration: 959; Percent complete: 95.9%; Average loss: 4.2697
Iteration: 960; Percent complete: 96.0%; Average loss: 3.8575
Iteration: 961; Percent complete: 96.1%; Average loss: 4.1998
Iteration: 962; Percent complete: 96.2%; Average loss: 4.1967
Iteration: 963; Percent complete: 96.3%; Average loss: 4.7055
Iteration: 964; Percent complete: 96.4%; Average loss: 4.9524
Iteration: 965; Percent complete: 96.5%; Average loss: 4.4818
Iteratio

# Evaluate model

In [27]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [31]:
evaluateInput(encoder, decoder, searcher, voc, 'I am mark')

Bot: i m you ? . . .
